In [3]:
import os
import csv
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
from utils import Static_dataGen, Dynamic_dataGen, Bonus_dataGen

key = os.environ.get("OPEN_AI_KEY")

# Static Dataset Creation


In [ ]:
staticDatagen = Static_dataGen(key)

no_of_StaticQuery_CompletionPairs2beGen = 10

data_dict = staticDatagen.genQuery(no_of_StaticQuery_CompletionPairs2beGen)

field_names= ['Query','Output']

with open('./datasets/Generated/saveStaticData.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(data_dict)

# Dynamic Dataset Creation

In [ ]:
dynamicDatagen = Dynamic_dataGen(key)

no_of_newTool2beAdded = 20

no_of_DynamicQuery_CompletionPairs2beGen = 10

dynamicDatagen.genDynamicTools(no_of_newTool2beAdded)

data_dict = dynamicDatagen.genDynamicQueryOutputPair(no_of_DynamicQuery_CompletionPairs2beGen)

field_names= ['Added_Tools','Query','Output']

with open('./datasets/Generated/saveDynamicData.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(data_dict)

# Bonus Dataset Creation

In [ ]:
bonusDatagen = Bonus_dataGen(key)

no_of_BonusQuery_CompletionPairs2beGen = 10

data_dict = bonusDatagen.genBonusQueryOutputPair(no_of_BonusQuery_CompletionPairs2beGen)

field_names= ['Query','Output']

with open('./datasets/Generated/saveBonusData.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(data_dict)

# Static Dataset Reading